In [ ]:
# https://blog.metaflow.fr/tensorflow-mutating-variables-and-control-flow-2181dd238e62


In [4]:
## Simple assign, called "manually" (with sess.run) multiple times 

import tensorflow as tf

# We define a Variable
x = tf.Variable(0, dtype=tf.int32)

# We use a simple assign operation
assign_op = tf.assign(x, x + 1)

sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())

for i in range(5):
    print('x:', sess.run(x))
    sess.run(assign_op)

sess.close()

x: 0
x: 1
x: 2
x: 3
x: 4


In [5]:
## Dynamic change of the shape of variable

import tensorflow as tf

# We define a "shape-able" Variable
x = tf.Variable(
    [], # A list of scalar
    dtype=tf.int32,
    validate_shape=False, # By "shape-able", i mean we don't validate the shape so we can change it
    trainable=False
)
# I build a new shape and assign it to x
concat = tf.concat([x, [0]], 0)
assign_op = tf.assign(x, concat, validate_shape=False) # We force TF, to skip the shape validation step

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(5):
    print('x:', sess.run(x), 'shape:', sess.run(tf.shape(x)))
    sess.run(assign_op)
sess.close()

x: [] shape: [0]
x: [0] shape: [1]
x: [0 0] shape: [2]
x: [0 0 0] shape: [3]
x: [0 0 0 0] shape: [4]


In [13]:
## "Naive" solution
# In order to calculate the returned variables assign_op does not have to be computed at all!! 

import tensorflow as tf

# We define our Variables and placeholders
x = tf.placeholder(tf.int32, shape=[], name='x')
y = tf.Variable(2, dtype=tf.int32)

# We set our assign op
assign_op = tf.assign(y, y + 1)

# We build our multiplication (this could be a more complicated graph)
out = x * y

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(3):
    y_, out_ = sess.run([y, out], feed_dict={x: 2})
    print('y:', y_)
    print('output:', out_)

sess.close()

y: 2
output: 4
y: 2
output: 4
y: 2
output: 4


In [16]:
## "Naive" solution
# In order to calculate the returned variables assign_op does not have to be computed at all!! 

import tensorflow as tf

# We define our Variables and placeholders
x = tf.placeholder(tf.int32, shape=[], name='x')
y = tf.Variable(2, dtype=tf.int32)

# We set our assign op
assign_op = tf.assign(y, y + 1)

# We build our multiplication, but this time, inside a control depedency scheme!
with tf.control_dependencies([assign_op]):
    # Now, we are under the dependency scope:
    # All the operations happening here will only happens after 
    # the "assign_op" has been computed first
    out = x * y

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(3):
    y_, out_ = sess.run([y, out], feed_dict={x: 2})
    print('y:', y_)
    print('output:', out_)

sess.close()

y: 3
output: 6
y: 4
output: 8
y: 5
output: 10


In [23]:
import tensorflow as tf

# I define a "shape-able" Variable
x = tf.Variable(
    [], 
    dtype=tf.int32,
    validate_shape=False, # By "shape-able", i mean we don't validate the shape
    trainable=False
)
# I build a new shape and assign it to x
concat = tf.concat([x, [0]], 0)
assign_op = tf.assign(x, concat, validate_shape=False)

with tf.control_dependencies([assign_op]):
    # I print x after the assignment
    x = tf.Print(x, data=[x, x.read_value()], message="x, x_read:")
    # The assign_op is called, but it seems that print statement happens
    # before the assignment, that is wrong.
    # https://www.tensorflow.org/api_docs/python/tf/Print
    # Note: Print currently not compatible with Jupyter Notebook :]
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(3):
        print(sess.run(x))

    sess.close()

#[]
#[0]
#[0 0]
#instead of:
#[0]
#[0 0]
#[0 0 0]
# !!!

[]
[0]
[0 0]
